In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment? Sentiment is required.
subject: What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.
price: How much did the customer pay? Use 'None' if no price was provided.

Format the output as JSON with the following keys:
sentiment
subject
price

text: {input}
"""

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

prompt_template = ChatPromptTemplate.from_template(template=TEMPLATE)
chain = prompt_template | llm
response = chain.invoke(input="I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.")

In [17]:
response.content

'```json\n{\n  "sentiment": "neutral",\n  "subject": "pizza",\n  "price": "None"\n}\n```'

Response Schemas

In [20]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="sentiment", description="is the text in a positive, neutral or negative sentiment? Sentiment is required."),
    ResponseSchema(name="subject", description="What subject is the text about? Use exactly one word. Use 'None' if no subject was provided."),
    ResponseSchema(name="price", description="How much did the customer pay? Use 'None' if no price was provided.", type="float")
]
print(response_schemas)

[ResponseSchema(name='sentiment', description='is the text in a positive, neutral or negative sentiment? Sentiment is required.', type='string'), ResponseSchema(name='subject', description="What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.", type='string'), ResponseSchema(name='price', description="How much did the customer pay? Use 'None' if no price was provided.", type='float')]


In [22]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // is the text in a positive, neutral or negative sentiment? Sentiment is required.
	"subject": string  // What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.
	"price": float  // How much did the customer pay? Use 'None' if no price was provided.
}
```


In [23]:
# Create prompt template
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Interprete the text and evaluate the text. "
            "sentiment: is the text in a positive, neutral or negative sentiment? "
            "subject: What subject is the text about? Use exactly one word. "
            "Just return the JSON, do not add ANYTHING, NO INTERPRETATION! "
            "text: {input}\n"
            "{format_instructions}\n"
        )
    ],
    input_variables=["input"],
    partial_variables={"format_instructions": format_instructions}
)

In [24]:
_input = prompt.format_prompt(input="I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.")
output = llm.invoke(_input.to_messages())
print(output.content)

```json
{
	"sentiment": "neutral",
	"subject": "pizza",
	"price": "None"
}
```


In [25]:
json_output = parser.parse(output.content)
print(json_output)

{'sentiment': 'neutral', 'subject': 'pizza', 'price': 'None'}


In [26]:
json_output.get("sentiment")

'neutral'